<a href="https://colab.research.google.com/github/shou01261/loto7/blob/main/loto7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sys
import os
import math
import logging
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
from torch.cuda.amp import GradScaler, autocast
import pandas as pd
import requests
from collections import Counter
import io
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
from collections import deque

# パフォーマンス設定
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True
torch.backends.cudnn.benchmark = True
torch.set_float32_matmul_precision('high')

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
def generate_ensemble_predictions(models, dataset, draw_number, num_preds=5, visualize=True, use_predicted=False):
    """
    複数のモデルを使用してアンサンブル予測を生成する関数

    Args:
        models (list): EnhancedLoto7Transformer モデルのリスト
        dataset (EnhancedLoto7Dataset): 使用するデータセット
        draw_number (int): 予測したい開催回
        num_preds (int): 各モデルから生成する予測の数
        visualize (bool): 予測結果を可視化するかどうか
        use_predicted (bool): 未来の開催回の場合に過去の予測結果を使用するかどうか

    Returns:
        list: アンサンブル予測の結果リスト (例: [当選数字7個のリスト, ...])
    """
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    all_predictions = []

    # 予測対象の開催回に対応する入力データを取得
    try:
        recent_src = dataset.get_recent_data_for_prediction(draw_number, use_predicted=use_predicted).unsqueeze(0).to(device)
    except ValueError as e:
        logger.error(f"Error getting data for prediction for draw {draw_number}: {str(e)}")
        return []

    # 各モデルで予測を実行
    for i, model in enumerate(models):
        model.eval() # 推論モード
        with torch.inference_mode(), torch.cuda.amp.autocast():
            logits = model(recent_src) # outputs: (batch, 7, 37)
            probs = F.softmax(logits, dim=-1) # (batch, 7, 37)

            # num_preds 個の予測を生成 (サンプリング)
            for _ in range(num_preds):
                sampled_numbers = []
                for pos in range(7):
                    # 確率分布から数字をサンプリング (重複あり)
                    sampled_idx = torch.multinomial(probs[0, pos], 1).item()
                    sampled_numbers.append(sampled_idx + 1) # 0-36を1-37に戻す
                all_predictions.append(sampled_numbers)

    # アンサンブル予測の統合 (例: 最も頻繁に出現する数字を選択)
    # 各位置 (1st, 2nd, ...) ごとに最も出現頻度の高い数字を選択
    ensemble_result = []
    for pos in range(7):
        pos_predictions = [pred[pos] for pred in all_predictions]
        # 最頻値を計算。同じ頻度の場合は数値の小さい方を選択（または別のルール）
        count = Counter(pos_predictions)
        most_common = count.most_common()
        if most_common:
            # 頻度が最大の候補をすべて取得
            max_freq = most_common[0][1]
            candidates = sorted([num for num, freq in most_common if freq == max_freq])
            ensemble_result.append(candidates[0]) # 最小の候補を選択
        else:
            # 予測がない場合はランダムな数字を選択 (例として)
            ensemble_result.append(np.random.randint(1, 38))

    logger.info(f"Ensemble prediction for draw {draw_number}: {ensemble_result}")

    # 可視化
    if visualize:
        plt.figure(figsize=(10, 6))
        all_numbers = [num for pred in all_predictions for num in pred]
        plt.hist(all_numbers, bins=range(1, 39), align='left', rwidth=0.8)
        plt.xticks(range(1, 38))
        plt.xlabel("Loto 7 Numbers")
        plt.ylabel("Frequency")
        plt.title(f"Number Distribution from Ensemble Predictions for Draw {draw_number}")
        plt.grid(axis='y', alpha=0.75)
        plt.show()

    return [ensemble_result] # 統合された予測結果をリストで返す
# ========================================
# データセット定義（未来予測機能追加版）
# ========================================
class EnhancedLoto7Dataset(Dataset):
    def __init__(self, csv_url, window_size=10, predict_future=False):
        self.window_size = window_size
        self.predict_future = predict_future
        self.prediction_history = deque(maxlen=window_size) if predict_future else None

        # データ取得と前処理
        response = requests.get(csv_url)
        response.encoding = 'shift_jis'
        df = pd.read_csv(io.StringIO(response.text))

        # 数字を整数として取得（1～37を0～36に変換）
        raw_draws = df[['第1数字','第2数字','第3数字','第4数字','第5数字','第6数字','第7数字']].values.astype(np.int64)
        self.draws_int = raw_draws - 1  # ターゲット（0～36）
        # 入力用は正規化（0～1）
        self.draws_norm = (raw_draws - 1) / 36.0

        # 開催回の正規化：各回の開催回を1次元の数値として扱う
        self.draw_numbers = df['開催回'].values
        self.min_draw_number = self.draw_numbers.min()
        self.max_draw_number = self.draw_numbers.max()
        self.draw_numbers_norm = (self.draw_numbers - self.min_draw_number) / (self.max_draw_number - self.min_draw_number)

        # サンプル生成：各サンプルは過去window_size回分のデータ
        # 各回の入力は、開催回（1次元）と宝くじ数字（7次元）の合計8次元
        self.samples = []
        for i in range(self.window_size, len(self.draws_int)):
            src = np.concatenate([
                self.draw_numbers_norm[i-self.window_size:i].reshape(-1, 1),  # shape: (window_size, 1)
                self.draws_norm[i-self.window_size:i]                    # shape: (window_size, 7)
            ], axis=1).flatten()  # フラットなベクトル（長さ = window_size * 8）
            tgt = self.draws_int[i]  # 当該回の7数字（整数 0～36 の配列）
            self.samples.append((torch.FloatTensor(src), torch.LongTensor(tgt)))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        return self.samples[idx]

    def add_prediction(self, draw_number, prediction):
        """未来予測用に予測結果を追加"""
        if not self.predict_future:
            return

        # 予測結果を正規化して保存 (predictionは1～37の数字)
        norm_pred = (np.array(prediction) - 1) / 36.0
        norm_draw = (draw_number - self.min_draw_number) / (self.max_draw_number - self.min_draw_number)

        self.prediction_history.append((norm_draw, norm_pred))

    def get_recent_data_for_prediction(self, draw_number, use_predicted=False):
        """
      指定された開催回に基づいて、過去のデータをウィンドウサイズ分取得する
      use_predicted=Trueの場合、過去の予測結果も使用する
      """
        # 開催回のインデックスを取得
        if draw_number not in self.draw_numbers:
            # 未来の開催回の場合、最後の開催回から増分を計算
            last_draw = self.draw_numbers[-1]
            if draw_number <= last_draw:
                raise ValueError(f"指定された開催回 {draw_number} はデータセット内に存在しません。")

            # 未来の開催回の場合、予測履歴を使用
            if not use_predicted or len(self.prediction_history) < self.window_size:
                # 予測履歴が不足している場合は実際のデータを使用
                idx = len(self.draw_numbers) - 1
            else:
                # 予測履歴からデータを構築
                pred_data = list(self.prediction_history)[-self.window_size:]
                src = np.concatenate([
                    np.array([d[0] for d in pred_data]).reshape(-1, 1),
                    np.array([d[1] for d in pred_data])
                ], axis=1).flatten()
                return torch.FloatTensor(src)
        else:
            idx = np.where(self.draw_numbers == draw_number)[0][0]

        # ウィンドウサイズ分の過去データが存在するか確認
        if idx < self.window_size:
            raise ValueError(f"指定された開催回 {draw_number} のインデックス {idx} はウィンドウサイズ {self.window_size} よりも小さいため、過去データが不足しています。")

        # 過去のデータをウィンドウサイズ分取得
        src = np.concatenate([
            self.draw_numbers_norm[idx-self.window_size:idx].reshape(-1, 1),  # shape: (window_size, 1)
            self.draws_norm[idx-self.window_size:idx]                    # shape: (window_size, 7)
        ], axis=1).flatten()  # フラットなベクトル（長さ = window_size * 8）
        return torch.FloatTensor(src)

# ========================================
# トレーナー定義（複数開催回予測機能追加版）
# ========================================
class AdvancedTrainer:
    def __init__(self, model, batch_size=256, grad_accum=2, predict_draw_numbers=None, model_idx=0):
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.model = model.to(self.device)
        self.predict_draw_numbers = predict_draw_numbers if predict_draw_numbers else []
        self.model_idx = model_idx # モデルのインデックスを追加

        # 標準のAdamWオプティマイザを使用
        self.optimizer = torch.optim.AdamW(
            model.parameters(),
            lr=2e-7,
            betas=(0.9, 0.98),
            weight_decay=1e-4
        )

        # 学習率スケジューラ
        self.scheduler = torch.optim.lr_scheduler.StepLR(self.optimizer, step_size=1, gamma=1.0001)
        self.min_lr = float('inf') # 追跡する最小学習率を初期化
        self.min_lr_epoch = -1 # 最小学習率を記録したエポック
        self.saved_best_lr_model = False # 最小LRでのモデルを一度だけ保存するためのフラグ
        # 省メモリ設定
        self.scaler = GradScaler(enabled=True)
        # 分類タスクのため CrossEntropyLoss を利用
        self.loss_fn = nn.CrossEntropyLoss()
        self.batch_size = batch_size
        self.grad_accum = grad_accum
        self.hist_loss = []
        self.hist_lr = []
        # 予測履歴はエポックごとのリストを格納
        # ファイル保存に切り替えるため、メモリ上には保持しないか、バッファリングのみ行う
        # self.hist_predictions = {draw_num: [] for draw_num in self.predict_draw_numbers}
    def train_epoch(self, epoch, loader, dataset=None):
        """
        単一のエポックのトレーニングを実行し、オプションで指定された開催回に対して予測を行う

        Args:
            epoch (int): 現在のエポック数
            loader (DataLoader): トレーニングデータのデータローダー
            dataset (EnhancedLoto7Dataset, optional): 予測に使用するデータセット。予測機能を使用しない場合はNone.
        """
        self.model.train() # モデルをトレーニングモードに設定
        total_loss = 0 # エポック全体の合計ロスを初期化
        self.optimizer.zero_grad() # 勾配をリセット

        # データローダーからバッチを取得し、トレーニングループを実行
        for i, (src, tgt) in enumerate(tqdm(loader, desc=f"Epoch {epoch}")):
            # データを適切なデバイス (CPU/GPU) に移動
            src = src.to(self.device, non_blocking=True)
            tgt = tgt.to(self.device, non_blocking=True)  # ターゲットの形状: (batch, 7)

            # 自動混合精度 (Autocast) を使用してフォワードパスを実行
            with autocast(dtype=torch.bfloat16):
                outputs = self.model(src)  # モデルの出力: (batch, 7, 37)
                # CrossEntropyLoss の入力形状に合わせるために reshape
                # 出力を (batch*7, 37), ターゲットを (batch*7,) に変換
                loss = self.loss_fn(outputs.view(-1, outputs.shape[-1]), tgt.view(-1)) / self.grad_accum

            # 勾配スケーリングを使用してバックワードパスを実行
            self.scaler.scale(loss).backward()

            # 勾配累積が設定されている場合、指定されたステップごとにオプティマイザーステップを実行
            if (i+1) % self.grad_accum == 0:
                self.scaler.step(self.optimizer) # スケーリングされた勾配でオプティマイザーステップ
                self.scaler.update() # 勾配スケーラーを更新
                self.optimizer.zero_grad() # 勾配をゼロにリセット
                # ここでのスケジューラーステップはバッチごとではなく、エポックの最後に変更することも検討
                # self.scheduler.step(epoch + i / len(loader)) # バッチごとのスケジューラーステップの例 (現在コメントアウト)

            # バッチのロスを合計ロスに加算 (勾配累積による正規化前のロスを加えることで平均を正しく計算)
            total_loss += loss.item() * self.grad_accum # grad_accum で割った分を元に戻す

        # エポックの平均ロスを計算
        avg_loss = total_loss / len(loader)
        # 現在の学習率を取得
        current_lr = self.scheduler.get_last_lr()[0] # エポック終了時点のLRを取得
        # ロスと学習率の履歴を記録
        self.hist_loss.append(avg_loss)
        self.hist_lr.append(current_lr) # LRをリストに追加
        # ロギング
        logger.info(f"Epoch {epoch} | Loss: {avg_loss:.4f} | LR: {current_lr:.9e}")

        # エポック終了後に学習率スケジューラーステップを適用
        # DataLoader のイテレーションが完了した後に一度だけステップを実行
        self.scheduler.step()

        # --- 未来予測機能 ---
        # 指定された開催回に対して予測を行い、結果をファイルに追記
# Inside AdvancedTrainer.train_epoch method, in the prediction block
        if self.predict_draw_numbers and dataset:
            self.model.eval()
            with torch.inference_mode(), torch.cuda.amp.autocast():
                num_preds_per_model = 5
                logger.info(f"Predicting for draw numbers: {self.predict_draw_numbers} at epoch {epoch} for model {self.model_idx}") # 予測対象の開催回をログ出力
                logger.info(f"Current working directory during prediction: {os.getcwd()}") # カレントディレクトリをログ出力

                for draw_num in self.predict_draw_numbers:
                    try:
                        use_predicted = draw_num > dataset.draw_numbers[-1]
                        logger.info(f"Attempting prediction for draw {draw_num} (use_predicted: {use_predicted})")

                        try:
                            recent_src = dataset.get_recent_data_for_prediction(draw_num, use_predicted=use_predicted).unsqueeze(0).to(self.device)
                            logits = self.model(recent_src)
                            probs = F.softmax(logits, dim=-1)

                            single_model_predictions_for_draw = []
                            for _ in range(num_preds_per_model):
                                sampled_numbers = []
                                for pos in range(7):
                                    sampled_idx = torch.multinomial(probs[0, pos], 1).item()
                                    sampled_numbers.append(sampled_idx + 1)
                                single_model_predictions_for_draw.append(sampled_numbers)

                            logger.info(f"Generated {len(single_model_predictions_for_draw)} predictions for draw {draw_num} at epoch {epoch} by model {self.model_idx}.")
                            if single_model_predictions_for_draw:
                                logger.info(f"Example prediction: {single_model_predictions_for_draw[0]}") # 生成された予測の例をログ出力
                            else:
                                logger.warning(f"No predictions generated for draw {draw_num} at epoch {epoch}.")


                        except ValueError as e:
                            logger.warning(f"Could not get data for prediction for draw {draw_num} at epoch {epoch}: {str(e)}. Skipping prediction for this draw.")
                            continue # この開催回に対するこれ以上の処理をスキップ
                        except Exception as data_e:
                            logger.error(f"An unexpected error occurred getting data for draw {draw_num}: {data_e}")
                            continue


                        # 予測結果をファイルに追記
                        history_file = f"model{self.model_idx}_{draw_num}_predictions.csv"
                        logger.info(f"Attempting to save predictions to: {history_file}")

                        # ファイルが存在しない場合はヘッダーが必要
                        header_needed = not os.path.exists(history_file)
                        logger.info(f"Header needed for {history_file}: {header_needed}")


                        # 生成された予測データがある場合のみ保存
                        if single_model_predictions_for_draw:
                            all_preds_str = "_".join([" ".join(map(str, pred)) for pred in single_model_predictions_for_draw])

                            header_row = ['Epoch', 'Loss', 'LR', 'All_Predictions']
                            row_to_save = [epoch, avg_loss, current_lr, all_preds_str]

                            df_history = pd.DataFrame([row_to_save], columns=header_row)

                            try:
                                # ファイルに保存 (存在しない場合はヘッダー付きで新規作成、存在する場合はヘッダーなしで追記)
                                if header_needed:
                                    df_history.to_csv(history_file, mode='w', header=True, index=False)
                                    logger.info(f"Saved predictions to {history_file} with header (mode='w').")
                                else:
                                    df_history.to_csv(history_file, mode='a', header=False, index=False)
                                    logger.info(f"Appended predictions to {history_file} (mode='a').")
                            except Exception as save_e:
                                logger.error(f"Error saving predictions to {history_file}: {save_e}")

                        else:
                            logger.warning(f"No predictions to save for draw {draw_num} at epoch {epoch} by model {self.model_idx} after generation check.")

                        # 未来の開催回に対する予測であり、かつ予測が生成された場合、データセットの履歴に最初の予測結果を追加
                        if use_predicted and single_model_predictions_for_draw:
                            try:
                                dataset.add_prediction(draw_num, single_model_predictions_for_draw[0])
                                logger.info(f"Added prediction for draw {draw_num} to dataset history for use_predicted.")
                            except Exception as add_e:
                                logger.error(f"Error adding prediction to dataset history for draw {draw_num}: {add_e}")

                    except Exception as overall_e:
                        # 予測または保存中に発生したその他のエラーを捕捉
                        logger.error(f"An overall error occurred during prediction or saving for draw {draw_num} at epoch {epoch}: {overall_e}")
# ========================================
# Loto7 Transformer モデル定義
# ========================================
class EnhancedLoto7Transformer(nn.Module):
    def __init__(self, window_size=10, d_model=128, nhead=8, num_layers=6, num_classes=37):
        super().__init__()
        self.window_size = window_size
        self.input_dim = 8  # 各時刻の入力は開催回1次元 + 宝くじ数字7次元

        # 開催回と宝くじ数字をそれぞれ別個に埋め込む
        self.draw_num_embed = nn.Linear(1, d_model // 4)
        self.lottery_embed = nn.Linear(7, d_model - d_model // 4)
        self.combined_proj = nn.Linear(d_model, d_model)

        self.rotary = EnhancedRotaryEmbedding(d_model)
        self.encoder_layers = nn.ModuleList([
            EnhancedTransformerEncoder(d_model, nhead) for _ in range(num_layers)
        ])
        self.fc_out = nn.Sequential(
            nn.Linear(d_model, d_model),
            nn.GELU(),
            nn.Linear(d_model, num_classes)
        )
        self.dropout = nn.Dropout(0.1)
        self._init_weights()

    def _init_weights(self):
         for p in self.parameters():
             if p.dim() > 1:
                 nn.init.xavier_uniform_(p)

    def forward(self, src):
         # src はフラットなベクトル（サイズ = window_size * input_dim）
         # (batch, window_size, input_dim) に変形
         src = src.view(-1, self.window_size, self.input_dim)
         # 開催回（1次元）と宝くじ数字（7次元）を分離
         draw_num = src[:, :, :1]       # shape: (batch, window_size, 1)
         lottery_nums = src[:, :, 1:]     # shape: (batch, window_size, 7)
         draw_emb = self.draw_num_embed(draw_num)      # (batch, window_size, d_model/4)
         lottery_emb = self.lottery_embed(lottery_nums)  # (batch, window_size, d_model - d_model/4)
         combined = torch.cat([draw_emb, lottery_emb], dim=-1)  # (batch, window_size, d_model)
         combined = self.combined_proj(combined)

         # ロータリエンベディングの適用
         pos = self.rotary(combined)
         combined = apply_rotary_pos_emb(pos, combined)
         combined = self.dropout(combined)

         for layer in self.encoder_layers:
              combined = layer(combined)

         # 最後の7タイムステップから各数字の予測を生成（出力 shape: (batch, 7, 37)）
         return self.fc_out(combined[:, -7:])

# ========================================
# メイン処理（未来予測対応版）
# ========================================
def main():
    # コマンドライン引数の解析
    mode = 'train'  # デフォルトはトレーニングモード
    predict_only = False
    predict_draw_numbers = []

    if len(sys.argv) > 1:
        if mode == 'predict':
            predict_only = True
            # 予測のみモードの場合、予測したい開催回を直接指定するか、入力から取得
            # ここでは仮に固定値を設定するか、必要に応じてユーザー入力を求めます
            predict_draw_numbers = [620, 621, 622] # 例
            # predict_draw_numbers = [int(input("予測したい開催回を入力してください: "))] # インタラクティブな場合
        elif mode == 'train':
             predict_draw_numbers = [620,621,622,623,624,625,626,627,628,629,630] # トレーニング中に予測する開催回

    # Load minimal data to get last draw number
    last_actual_draw = 0
    try:
        response = requests.get("https://loto7.thekyo.jp/data/loto7.csv")
        response.encoding = 'shift_jis'
        df_temp = pd.read_csv(io.StringIO(response.text))
        last_actual_draw = df_temp['開催回'].values[-1]
        del df_temp # Free up memory
    except Exception as e:
        logger.error(f"Failed to load initial data to check last draw number: {e}")

    # Determine predict_future based on whether any requested draw number is beyond the last actual draw
    should_predict_future = predict_draw_numbers and any(num > last_actual_draw for num in predict_draw_numbers)

    # データセット初期化（未来予測モードで初期化）
    dataset = EnhancedLoto7Dataset(
        "https://loto7.thekyo.jp/data/loto7.csv",
        window_size=10,
        predict_future=should_predict_future
    )

    if predict_only:
        # 予測のみモード
        models = []
        for i in range(3): # 3モデルをロードしようとする
            model_path = f"loto7_enhanced_epoch1000_model{i}.pth"
            if os.path.exists(model_path):
                model = EnhancedLoto7Transformer(window_size=10)
                model.load_state_dict(torch.load(model_path, map_location=torch.device('cuda' if torch.cuda.is_available() else 'cpu')))
                model.eval()
                models.append(model)
            else:
                logger.warning(f"Model file not found: {model_path}. Skipping model {i}.")

        if not models:
             logger.error("予測に使えるモデルが見つかりませんでした。")
             sys.exit(1)

        # 各開催回に対して予測
        for draw_num in predict_draw_numbers:
            try:
                # 未来の開催回の場合は予測結果を使用
                use_predicted = draw_num > dataset.draw_numbers[-1]
                predictions = generate_ensemble_predictions(
                    models, dataset, draw_num,
                    num_preds=5,
                    visualize=True,
                    use_predicted=use_predicted
                )

                # 結果を保存
                if predictions: # Only save if predictions were generated (e.g., not empty list from error)
                    # アンサンブル予測結果をCSVとして保存
                    pd.DataFrame(predictions, columns=[f'Num{j+1}' for j in range(7)]).to_csv(f"predictions_{draw_num}.csv", index=False)
                    logger.info(f"Ensemble prediction results for draw {draw_num} saved")

                    # 未来の開催回の場合は予測結果をデータセットに追加
                    # generate_ensemble_predictionsは[result]を返すため、predictions[0]を使用
                    if use_predicted and predictions[0]:
                         dataset.add_prediction(draw_num, predictions[0])
                else:
                    logger.warning(f"No predictions generated for draw {draw_num}, skipping save and history add.")

            except Exception as e:
                logger.error(f"Prediction failed for draw {draw_num}: {str(e)}")
        return

    # トレーニングモード
    # ハイパフォーマンスデータローダー
    loader = DataLoader(
        dataset,
        batch_size=512,
        shuffle=True,
        num_workers=os.cpu_count(),
        pin_memory=True,
        persistent_workers=True
    )

    # モデルとトレーナーの作成
    models = [EnhancedLoto7Transformer(window_size=10) for _ in range(3)]
    # トレーナーにモデルのインデックスを渡す
    trainers = [AdvancedTrainer(model, predict_draw_numbers=predict_draw_numbers, model_idx=i) for i, model in enumerate(models)]

    # トレーニング実行
    num_epochs = 100001
    save_interval = 1000

    for epoch in range(1, num_epochs):
        for trainer in trainers:
            # Pass the dataset to train_epoch for prediction feature
            trainer.train_epoch(epoch, loader, dataset)

        # モデル保存 (既存のロジック、epoch % save_interval == 0 で保存)
        if epoch % save_interval == 0:
            for i, model in enumerate(models):
                try:
                    torch.save(model.state_dict(), f"loto7_epoch{epoch}_model{i}.pth")
                    logger.info(f"Model loto7_enhanced_epoch{epoch}_model{i}.pth saved.")
                except Exception as e:
                    logger.error(f"Failed to save model epoch {epoch} model {i}: {e}")

        for i, model in enumerate(models):
           torch.save(model.state_dict(), f"loto7_model{i}.pth")

# Placeholder classes (Retained for completeness, replace with your actual implementations)
class EnhancedRotaryEmbedding(nn.Module):
    def __init__(self, dim, base=10000):
        super().__init__()
        inv_freq = 1. / (base ** (torch.arange(0, dim, 2).float() / dim))
        self.register_buffer("inv_freq", inv_freq)

    def forward(self, x):
        seq_len = x.shape[1]
        t = torch.arange(seq_len, device=x.device).type_as(self.inv_freq)
        freqs = torch.einsum('i,j->ij', t, self.inv_freq)
        emb = torch.cat((freqs, freqs), dim=-1)
        return emb

def apply_rotary_pos_emb(pos, qk):
    if qk.ndim == 4: # For QK (batch, heads, seq_len, head_dim)
        seq_len = qk.shape[2]
        # Placeholder rotation - replace with actual RoPE rotation
        pos_expanded = pos.unsqueeze(1).unsqueeze(1).expand_as(qk)
        return qk + pos_expanded # Simplified addition, not real RoPE rotation
    elif qk.ndim == 3: # For combined input (batch, seq_len, dim)
         seq_len = qk.shape[1]
         # Placeholder rotation - replace with actual RoPE rotation
         pos_expanded = pos.unsqueeze(0).expand_as(qk)
         return qk + pos_expanded # Simplified addition, not real RoPE rotation
    else:
        raise ValueError("Unsupported input dimensions for apply_rotary_pos_emb")


class EnhancedTransformerEncoder(nn.Module):
    def __init__(self, d_model, nhead):
        super().__init__()
        self.norm1 = nn.LayerNorm(d_model)
        self.attn = nn.MultiheadAttention(d_model, nhead, batch_first=True)
        self.norm2 = nn.LayerNorm(d_model)
        self.mlp = nn.Sequential(
            nn.Linear(d_model, d_model * 4),
            nn.GELU(),
            nn.Linear(d_model * 4, d_model)
        )
        self.dropout1 = nn.Dropout(0.1)
        self.dropout2 = nn.Dropout(0.1)

    def forward(self, src):
        x = self.norm1(src)
        attn_output, _ = self.attn(x, x, x)
        x = src + self.dropout1(attn_output)
        x = x + self.dropout2(self.mlp(self.norm2(x)))
        return x


class EnhancedRotaryEmbedding(nn.Module):
    def __init__(self, dim, base=10000):
        super().__init__()
        # Placeholder: Your actual Rotary Embedding implementation
        # Example:
        inv_freq = 1. / (base ** (torch.arange(0, dim, 2).float() / dim))
        self.register_buffer("inv_freq", inv_freq)

    def forward(self, x):
        # Placeholder: Apply RoPE
        # x shape: (batch, seq_len, dim)
        seq_len = x.shape[1]
        t = torch.arange(seq_len, device=x.device).type_as(self.inv_freq)
        freqs = torch.einsum('i,j->ij', t, self.inv_freq)
        emb = torch.cat((freqs, freqs), dim=-1)
        return emb # Returning embedding to be used by apply_rotary_pos_emb

def apply_rotary_pos_emb(pos, qk):
    # Placeholder: Apply Rotary Position Embedding
    # pos shape: (seq_len, dim)
    # qk shape: (batch, num_heads, seq_len, head_dim) or (batch, seq_len, dim)
    # Your actual implementation of applying RoPE rotation
    # Example (simplified):
    if qk.ndim == 4: # For QK (batch, heads, seq_len, head_dim)
        seq_len = qk.shape[2]
        pos = pos.unsqueeze(1).unsqueeze(1) # (seq_len, 1, 1, dim)
        return qk + pos # This is NOT how RoPE works, replace with actual rotation logic
    elif qk.ndim == 3: # For combined input (batch, seq_len, dim)
         seq_len = qk.shape[1]
         pos = pos.unsqueeze(0) # (1, seq_len, dim)
         return qk + pos # This is NOT how RoPE works, replace with actual rotation logic
    else:
        raise ValueError("Unsupported input dimensions for apply_rotary_pos_emb")


class EnhancedTransformerEncoder(nn.Module):
    def __init__(self, d_model, nhead):
        super().__init__()
        # Placeholder: Your actual Transformer Encoder layer
        # Example using standard PyTorch TransformerEncoderLayer:
        # from torch.nn import TransformerEncoderLayer
        # self.layer = TransformerEncoderLayer(d_model, nhead, dim_feedforward=d_model*4, dropout=0.1, batch_first=True)
        # Replace with your custom implementation if needed
        self.norm1 = nn.LayerNorm(d_model)
        self.attn = nn.MultiheadAttention(d_model, nhead, batch_first=True)
        self.norm2 = nn.LayerNorm(d_model)
        self.mlp = nn.Sequential(
            nn.Linear(d_model, d_model * 4),
            nn.GELU(),
            nn.Linear(d_model * 4, d_model)
        )
        self.dropout1 = nn.Dropout(0.1)
        self.dropout2 = nn.Dropout(0.1)

    def forward(self, src):
        # Placeholder: Transformer Encoder forward pass
        # x = self.layer(src) # Using standard layer
        # Custom layer forward:
        x = self.norm1(src)
        # MultiheadAttention expects query, key, value. For self-attention, they are the same.
        # It also needs a mask if sequence length varies, but here it's fixed window_size.
        # Check return format: (output, attn_output_weights)
        attn_output, _ = self.attn(x, x, x)
        x = src + self.dropout1(attn_output)
        x = x + self.dropout2(self.mlp(self.norm2(x)))
        return x


if __name__ == "__main__":
    main()

<ipython-input-1-73cea70b3692>:218: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = GradScaler(enabled=True)
/usr/local/lib/python3.11/dist-packages/torch/amp/grad_scaler.py:132: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(
Epoch 1:   0%|          | 0/10 [00:00<?, ?it/s]<ipython-input-1-73cea70b3692>:248: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(dtype=torch.bfloat16):
/usr/local/lib/python3.11/dist-packages/torch/amp/autocast_mode.py:266: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(
Epoch 1: 100%|██████████| 10/10 [00:02<00:00,  4.86it/s]
<ipython-input-1-73cea70b3692>:286: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', a

KeyboardInterrupt: 

In [ ]:
!pip install apex transformers datasets tokenizers
!pip install pytorch

[1, 6, 13, 19, 25, 21, 37] _
[2, 4, 8, 16, 19, 36, 32] _
[1, 6, 11, 16, 25, 26, 35] _
[1, 10, 8, 16, 27, 24, 35] _
[1, 7, 15, 16, 27, 31, 35]

In [ ]:
!sudo apt-get update
!sudo apt-get install -y libstdc++6
!sudo apt-get install -y gcc-12 g++-12
!sudo update-alternatives --install /usr/bin/gcc gcc /usr/bin/gcc-12 100
!sudo update-alternatives --install /usr/bin/g++ g++ /usr/bin/g++-12 100
!pip uninstall bitsandbytes
!pip install bitsandbytes
!pip uninstall deepspeed
!pip install deepspeed